In [170]:
seq_loca = dict()
seq_date = dict()
f_csv = open('../data/2019nCoV_genomes.2020_04_14.csv', 'r')
f_csv.readline()
for line in f_csv:
    tokens = line.replace(' ', '_').strip().split(',')
    seq_id = tokens[0]

    collect_date = tokens[-1]
    if len(collect_date) == 7:
        collect_date += '-00'
    geo_location = tokens[4]
    geo_location = geo_location.replace('North_Carolina', 'NC')
    geo_location = geo_location.replace('Illinois', 'IL')
    geo_location = geo_location.replace('San_Francisco', 'Ca-SF')
    geo_location = geo_location.replace('"', '')
    geo_location = geo_location.replace('_','')
    geo_location = geo_location.replace('SnohomishCounty','Snohomish_Wa')
    seq_loca[seq_id]=geo_location
    seq_date[seq_id]=collect_date
f_csv.close()
seq_list = dict()
f_fa = open('../data/2019nCoV_genomes.2020_04_14.fa', 'r')
for line in f_fa:
    if line.startswith('>'):
        tmp_h = line.strip().lstrip('>').split()[0]
        seq_list[tmp_h] = []
    else:
        seq_list[tmp_h].append(line.strip())
f_fa.close()
ofile = open("20171224USAfastafile.fa", "w")
for idea,location in seq_loca.items():
    if location.find('USA')!=-1:
        ofile.write(">"+idea+'|'+seq_loca[idea]+'|'+seq_date[idea]+'\n'+''.join(seq_list[idea])+'\n')
ofile.close()

In [8]:
aligned_info=open('20200505 MSA.txt','r')
aligned_ext_info=dict()
aligned_seq_info=dict()
for line in aligned_info:
    if line.startswith('>'):
        tmp_h = line.strip().lstrip('>').split('|')
        aligned_ext_info[tmp_h[0]] = [tmp_h[2],tmp_h[1]]
        aligned_seq_info[tmp_h[0]] = []
    else:
        line=line.replace('n','-')
        line=line.replace('y','-')
        line=line.replace('s','-')
        line=line.replace('r','-')
        line=line.replace('w','-')
        line=line.replace('h','-')
        aligned_seq_info[tmp_h[0]].append(line.strip())
aligned_info.close()

In [7]:
def identify_region_date(name):
    a=[]
    for key, value in aligned_ext_info.items():
        if value.count(name)!=0:
            a.append(key)
    return a

In [9]:
def comparison_list(genome2):
    n=len(genome2)
    positions = [[] for _ in range(n)]
    elements = [[] for _ in range(n)]
    info=dict()
    for i in range(len(genome2)):
        genome1=''.join(aligned_seq_info['NC_045512.2'])
        ahaha=''.join(aligned_seq_info[genome2[i]])
        if (len(genome1) == len(ahaha)):
            for j in range(len(genome1)):
                if genome1[j]!='-' and ahaha[j]!='-':
                    if genome1[j]!=ahaha[j]:
                        bc=str(genome1[j]+ahaha[j])
                        positions[i].append(j)
                        elements[i].append(bc)
                        info[j]=bc
        elif (len(genome1) > len(ahaha)):
            for j in range(len(ahaha)):
                if genome1[j]!='-' and ahaha[j]!='-':
                    if genome1[j]!=ahaha[j]:
                        bc=str(genome1[j]+ahaha[j])
                        positions[i].append(j)
                        elements[i].append(bc)
                        info[j]=bc
        elif (len(genome1) < len(ahaha)):
            for j in range(len(genome1)):
                if genome1[j]!='-' and ahaha[j]!='-':
                    if genome1[j]!=ahaha[j]:
                        bc=str(genome1[j]+ahaha[j])
                        positions[i].append(j)
                        elements[i].append(bc)
                        info[j]=bc
    return positions,elements,info    

In [10]:
WA_list=identify_region_date('USA:WA')
MA_list=identify_region_date('USA:MA')
PA_list=identify_region_date('USA:PA')
NY_list=identify_region_date('USA:NY')
group_1=['MT233526','MT044258','MT027062','MN988713','MN994467','MT106054','MT106053','MT188340','MT159705','MT159722','MT276325','MT276329','MT304484','MT304485','MT325632']
group_2=['MT325599','MT293219','MT259250','MT258379','MT308693','MT325590','MT263433','MT263411','MT304490','MT263409','MT293214','MT293220','MT263389','MT293206','MT263420']

In [5]:
#Q2
def mutation_rate_1(group1,positions):
    length=[]
    for i in range(len(positions)):
        cd=len(positions[i])
        length.append(cd)
    set_length=list(set(length))
    length_group=[[] for _ in range(len(set_length))]
    time_group=[[] for _ in range(len(set_length))]
    time_assemble=[[] for _ in range(len(set_length))]
    mutation_rate=[[] for _ in range(len(set_length)-1)]
    for j in range(len(set_length)):
        for k in range(len(length)):
             if set_length[j]==length[k]:
                length_group[j].append(group1[k])
    for l in range(len(set_length)):
        for u in range(len(length_group[l])):
            abcd=length_group[l][u]
            time_group[l].append(aligned_ext_info[abcd][0])
        ab=min(time_group[l])
        time_assemble[l].append(ab)
    from datetime import date
    for s in range(len(set_length)-1):
        abc=''.join(time_assemble[s]).split('-')
        de=''.join(time_assemble[s+1]).split('-')
        if len(abc[1])>1 and abc[1].find('0')!=-1:
            month1 = abc[1][-1]   
        if len(de[1])>1 and de[1].find('0')!=-1:
            month2=de[1][-1]
        if int(abc[2])!=0 and int(de[2])!=0:
            c0=date(int(abc[0]),int(month1),int(abc[2]))
            d0=date(int(de[0]),int(month2),int(de[2]))
            delta = d0-c0
            if delta.days>0:
                abv=int(set_length[s+1])-int(set_length[s])
                mutation_rate[s].append(abv/delta.days)
        
    return set_length,time_assemble,mutation_rate

In [6]:
def recurring_position(position1):
    for i in range(len(position1)-1):
        position1[0].extend(position1[i+1])
    a1=position1[0]
    
    from collections import Counter
    Counter(a1)    
    return Counter(a1)

In [11]:
position_1,element_1,info1=comparison_list(group_1)
position_2,element_2,info2=comparison_list(group_2)
position_3,element_3,info3=comparison_list(WA_list)
position_4,element_4,info4=comparison_list(MA_list)
position_5,element_5,info5=comparison_list(PA_list)
position_6,element_6,info6=comparison_list(NY_list)
set_length_5,time5,mutation_rate5=mutation_rate_1(PA_list,position_5)
set_length_6,time6,mutation_rate6=mutation_rate_1(NY_list,position_6)

In [12]:
from __future__ import print_function
c1=recurring_position(position_1)
c2=recurring_position(position_2)
c3=recurring_position(position_3)
c4=recurring_position(position_4)
c2.update(c1)
c3.update(c2)
c4.update(c3)
info2.update(info1)
info3.update(info2)
info4.update(info3)
info5.update(info4)
info6.update(info5)
print(info6)

{344: 'ct', 1291: 'ct', 3269: 'ct', 14640: 'ct', 23638: 'ag', 25798: 'gt', 2648: 'ct', 19109: 'ct', 18738: 'gk', 20079: 'ga', 28715: 'at', 20500: 'ag', 29969: 'gc', 15037: 'ct', 17479: 'tc', 26379: 'gt', 3750: 'gt', 17655: 'ag', 24269: 'ct', 29089: 'ct', 3493: 'cm', 26344: 'gt', 26468: 'gt', 9000: 'ga', 9014: 'ct', 17979: 'ct', 18090: 'ag', 18292: 'ct', 27760: 'ag', 28379: 'tc', 20513: 'tc', 34: 'at', 35: 'ct', 0: 'ac', 2: 'tc', 3: 'at', 4: 'at', 5: 'at', 15068: 'ag', 322: 'gt', 30102: 'ta', 30103: 'gv', 33: 'at', 36: 'ca', 30104: 'at', 28943: 'ct', 2864: 'gt', 11315: 'gt', 27619: 'tc', 30105: 'ca', 8834: 'ga', 26387: 'ga', 13457: 'ct', 11936: 'ct', 20488: 'tc', 1064: 'ct', 1777: 'ct', 9169: 'ct', 1: 'ta', 30115: 'ac', 2342: 'ct', 21810: 'ct', 149: 'tc', 152: 'tg', 11906: 'ct', 29061: 'ct', 4187: 'gt', 8913: 'ag', 22839: 'ga', 10098: 'ct', 25572: 'gc', 3638: 'ac', 6016: 'ct', 23760: 'ct', 54: 'ac', 8310: 'ct', 17558: 'ct', 18116: 'ag', 19877: 'gt', 2144: 'ct', 22085: 'gt', 23223: 'ga',

In [13]:
UTR_5=[]
ORF1ab=[]
S=[]
ORF3a=[]
E=[]
M=[]
ORF6=[]
ORF7a=[]
ORF7b=[]
ORF8=[]
N=[]
ORF10=[]
UTR_3=[]
for key,value in c4.items():
    if int(key)<265:
        UTR_5.append(c4[key])
    elif int(key)<21555:
        ORF1ab.append(c4[key])
    elif int(key)<25384:
        S.append(c4[key])
    elif int(key)<26220:
        ORF3a.append(c4[key])
    elif int(key)<26472:
        E.append(c4[key])
    elif int(key)<27191:
        M.append(c4[key])
    elif int(key)<27387:
        ORF6.append(c4[key])
    elif int(key)<27759:
        ORF7a.append(c4[key])
    elif int(key)<27887:
        ORF7b.append(c4[key])
    elif int(key)<28259:
        ORF8.append(c4[key])
    elif int(key)<29533:
        N.append(c4[key])
    elif int(key)<29674:
        ORF10.append(c4[key])
    else:
        UTR_3.append(c4[key])

In [17]:
UTR_5=[]
ORF1ab=[]
S=[]
ORF3a=[]
E=[]
M=[]
ORF6=[]
ORF7a=[]
ORF7b=[]
ORF8=[]
N=[]
ORF10=[]
UTR_3=[]
for key,value in info6.items():
    if int(key)<265:
        UTR_5.append(info6[key])
    elif int(key)<21555:
        ORF1ab.append(info6[key])
    elif int(key)<25384:
        S.append(info6[key])
    elif int(key)<26220:
        ORF3a.append(info6[key])
    elif int(key)<26472:
        E.append(info6[key])
    elif int(key)<27191:
        M.append(info6[key])
    elif int(key)<27387:
        ORF6.append(info6[key])
    elif int(key)<27759:
        ORF7a.append(info6[key])
    elif int(key)<27887:
        ORF7b.append(info6[key])
    elif int(key)<28259:
        ORF8.append(info6[key])
    elif int(key)<29533:
        N.append(info6[key])
    elif int(key)<29674:
        ORF10.append(info6[key])
    else:
        UTR_3.append(info6[key])
print(UTR_5)
print(ORF1ab)
print(S)
print(ORF3a)
print(E)
print(M)
print(ORF6)
print(ORF7a)
print(ORF7b)
print(ORF8)
print(N)
print(ORF10)
print(UTR_3)

['at', 'ct', 'ac', 'tc', 'at', 'at', 'at', 'at', 'ca', 'ta', 'tc', 'tg', 'ac', 'ga', 'gc', 'ta', 'ct', 'gc', 'ag', 'at', 'ca', 'at', 'tg', 'ct', 'tg', 'gt', 'tc', 'gc', 'ct', 'tc', 'tc']
['ct', 'ct', 'ct', 'ct', 'ct', 'ct', 'gk', 'ga', 'ag', 'ct', 'tc', 'gt', 'ag', 'cm', 'ga', 'ct', 'ct', 'ag', 'ct', 'tc', 'ag', 'gt', 'gt', 'gt', 'ga', 'ct', 'ct', 'tc', 'ct', 'ct', 'ct', 'ct', 'ct', 'gt', 'ag', 'ct', 'ac', 'ct', 'ct', 'ct', 'ag', 'gt', 'ct', 'ga', 'ct', 'ct', 'ct', 'tg', 'tc', 'ct', 'tc', 'ct', 'ct', 'ct', 'ag', 'ag', 'ag', 'tc', 'ct', 'tc', 'ag', 'ga', 'gt', 'ct', 'ct', 'ct', 'ct', 'ct', 'tc', 'ta', 'ct', 'ct', 'ct', 'ct', 'ct', 'gt', 'ct', 'ct', 'ct', 'ct', 'gt', 'tc', 'gc', 'ag', 'ct', 'ct', 'tc', 'ag', 'ct', 'ct', 'ct', 'ga', 'tc', 'ct', 'ga', 'gt', 'ct', 'ct', 'ct', 'ct', 'ga', 'ga', 'ga', 'ac', 'gt', 'ct', 'tc', 'gt', 'ag', 'at', 'tc', 'ct', 'gt', 'ct', 'ct', 'ga', 'ct', 'ct', 'ga', 'ca', 'ct', 'tc', 'ct', 'ct', 'ct', 'tc', 'ga', 'ct', 'ag', 'tc', 'ct', 'ct', 'gt', 'tc', 'ta', 't